### The Battle of Neighborhoods (Week 1+Week2)

In [ ]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: | 

###### For this week, you will required to submit the following:

1 Introduction where you discuss the business problem and who would be interested in this project.

2 Data where you describe the data that will be used to solve the problem and the source of the data.

3 Methodology section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, if any, and what machine learnings were used and why.

4 Results section where you discuss the results.

5 Discussion section where you discuss any observations you noted and any recommendations you can make based on the results.

6 Conclusion section where you conclude the report.

#### 1. Introduction

suitable apartment for rent in Manhattan NY that complies with the demands on location, price and venues. The data required to resolve this challenge is described 

##### 2.Data Section

following data is required to answer the issues of the problem:

List of Boroughs and neighborhoods of Manhattan with their latitude and longitude

List of Subway metro stations in Manhattan with their relative location

Venues for each Manhattan neighborhoods ( that can be clustered)

Venues for subway metro stations

The data will be used as follows:

Use foursquare and geopy data to map the location of subway metro stations , separately and on top of the above clustered map in order to be able to identify the venues and amenities near each metro station, or explore each subway location separately

Addresses from rental locations will be converted to geodata( lat, long) using Geopy-distance and Nominatim.
Data will be searched in open data sources if available, from real estate sites if open to reading, libraries or other government agencies such as Metro New York

The procesing of these DATA will allow to answer the key questions to make a decision:

How venues distribute among Manhattan neighborhoods and around metro stations?

Are there tradeoffs between size and price and location?

Any other interesting statistical data findings of the real estate and overall data.

In [3]:
# Riverside Quay Southbank (Riverside Quay) Southbank VIC 3006 Australia
address = 'Riverside Quay, Southbank'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Melbourne home are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


The geograpical coordinate of Melbourne home are -37.8210248, 144.965629.


In [4]:
neighborhood_latitude=-37.8212898
neighborhood_longitude=144.9644173

In [6]:
LIMIT = 100 
radius = 500 
CLIENT_ID = 'CETGLD3C3BHQLWQYAVNIGTTHRNOCSMO3GDQ1KYN3V0RPQZVA' # My Foursquare ID
CLIENT_SECRET = 'X5CYNQD03LMAYUQNXYVHWUSI3D24LHWL1MN12OVWN5HKJP3K' # My Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CETGLD3C3BHQLWQYAVNIGTTHRNOCSMO3GDQ1KYN3V0RPQZVA
CLIENT_SECRET:X5CYNQD03LMAYUQNXYVHWUSI3D24LHWL1MN12OVWN5HKJP3K


In [7]:
url= 'https://api.foursquare.com/v2/venues/explore?&client_id=CETGLD3C3BHQLWQYAVNIGTTHRNOCSMO3GDQ1KYN3V0RPQZVA&client_secret=X5CYNQD03LMAYUQNXYVHWUSI3D24LHWL1MN12OVWN5HKJP3K&v=20180604&ll=-37.8212898,144.9644173&radius=500&limit=100'
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=CETGLD3C3BHQLWQYAVNIGTTHRNOCSMO3GDQ1KYN3V0RPQZVA&client_secret=X5CYNQD03LMAYUQNXYVHWUSI3D24LHWL1MN12OVWN5HKJP3K&v=20180604&ll=-37.8212898,144.9644173&radius=500&limit=100'

In [11]:
# create map of Southbank Melbourne using latitude and longitude values
map_sb = folium.Map(location=[latitude, longitude], zoom_start=20)

# add markers to map
for lat, lng, label in zip(SBnearby_venues['lat'], SBnearby_venues['lng'], SBnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_sb)  
    
map_sb

In [8]:
results = requests.get(url).json()
#results

In [9]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [10]:
venues = results['response']['groups'][0]['items']
    
SBnearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SBnearby_venues =SBnearby_venues.loc[:, filtered_columns]

# filter the category for each row
SBnearby_venues['venue.categories'] = SBnearby_venues.apply(get_category_type, axis=1)

# clean columns
SBnearby_venues.columns = [col.split(".")[-1] for col in SBnearby_venues.columns]

SBnearby_venues.head(10)

,name,categories,lat,lng
0,Southbank Promenade,Pedestrian Plaza,-37.819959,144.965467
1,Ponyfish Island,Bar,-37.819918,144.965021
2,Yarra River,River,-37.819684,144.965115
3,The Langham,Hotel,-37.820370,144.965710
4,Broad Bean Organic Grocer,Grocery Store,-37.822588,144.966912
5,Pure South,Australian Restaurant,-37.820232,144.965259
6,Arbory Bar & Eatery,Bar,-37.819020,144.966004
7,Arbory Afloat,Bar,-37.819430,144.966338
8,Candela Nuevo,Cocktail Bar,-37.818614,144.962604
9,Crown Promenade,Plaza,-37.821669,144.960354
